In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# seasons = ['f24', 's24', 'f23', 's23', 'f22','s22']
seasons = ['s24']

# df_races = pd.DataFrame()
# try:
#   df_races = pd.read_csv("racesf24s24f23.csv")
# except:
df_races = pd.DataFrame(columns=["Score", "Sailor","Link", "GradYear", "Position", "Partner", "Venue", "Regatta", "Scoring", "raceID", "Date", "Week", "Team", "Teamlink"])

regattas = {}
for season in seasons:
  url = f"https://scores.collegesailing.org/{season}/"
  page = requests.get(url)
  listSoup = BeautifulSoup(page.content, 'html.parser')
  
  tbody = listSoup.find('table', class_="season-summary").find('tbody')
  
  for link in tbody.find_all("a", href=True):
    if (season + "/" + link['href']) not in df_races['Regatta'].unique():
      scoring = link.parent.next_sibling.next_sibling.next_sibling.text
      if (scoring == "Team"):
        regattas[season + "/" + link['href']] = {"link":season + "/" + link['href'], "scoring":scoring}

In [52]:
regattaSoups = {}

for i, regatta in enumerate(list(regattas.values())):
    betterVenue = list(regattas.keys())[i]
    print(f"({i + 1}/{len(list(regattas.values()))}) getting soup for {betterVenue}")
    # all races
    url = f"https://scores.collegesailing.org/{regatta['link']}/all/"
    page = requests.get(url)
    allRaces = BeautifulSoup(page.content, 'html.parser')

    # sailors
    url = f"https://scores.collegesailing.org/{regatta['link']}/sailors/"
    page = requests.get(url)
    sailors = BeautifulSoup(page.content, 'html.parser')
    
    url = f"https://scores.collegesailing.org/{regatta['link']}/"
    page = requests.get(url)
    report = BeautifulSoup(page.content, 'html.parser')
    
    regattaSoups[betterVenue] = {"allRaces": allRaces, "sailors": sailors, 'report': report, "scoring": regatta['scoring']}

(1/73) getting soup for s24/icsa-open-team-racing-national
(2/73) getting soup for s24/icsa-women-team-race-national
(3/73) getting soup for s24/b10-team-race
(4/73) getting soup for s24/mcsa-women-team-race
(5/73) getting soup for s24/saisa-spring-women-team-race-championships
(6/73) getting soup for s24/2024-legler
(7/73) getting soup for s24/carter-ford-open-pccsc-team-race
(8/73) getting soup for s24/maisa-women-team-race-championships
(9/73) getting soup for s24/thompson-camel-site
(10/73) getting soup for s24/thompson-cga
(11/73) getting soup for s24/mcsa-open-team-race-qualifier
(12/73) getting soup for s24/saisa-spring-team-race-championships
(13/73) getting soup for s24/prosser
(14/73) getting soup for s24/neisa-open-team-race-fowle
(15/73) getting soup for s24/pccsc-women-team-race
(16/73) getting soup for s24/seisa-women-team-race
(17/73) getting soup for s24/seisa-open-team-racing
(18/73) getting soup for s24/yale-women-team-race
(19/73) getting soup for s24/mystic-lake-tea

In [283]:
def makeRaceList(raceRows):
    raceList = []
    round = ''
    
    # loop through races
    for raceRow in raceRows: 
        if raceRow.has_attr('class') and raceRow.attrs['class'][0] == 'roundrow':
            round = raceRow.text
            continue
        
        raceData = [el.text for el in raceRow.find_all('td')]
        raceNum = raceData[0]
        
        teamAEl = raceRow.find_all('td', class_='team1')
        if(len(teamAEl) < 3):
            print("No score for this race?", raceNum)
            continue
        teamAName = list(teamAEl[1].stripped_strings)[0]
        teamANick = list(teamAEl[1].stripped_strings)[1]
        teamALink = teamAEl[1].find('a').attrs['href']
        teamAScore = teamAEl[2].text
        teamAWin = teamAEl[1].attrs['class'][0].split("-")[1]
        teamA = {'name': teamAName, 'nick': teamANick, 'link': teamALink.split("/")[2], 'outcome': teamAWin, 'score': teamAScore}
            
        teamBEl = raceRow.find_all('td', class_='team2')
        teamBName = list(teamBEl[1].stripped_strings)[0]
        teamBNick = list(teamBEl[1].stripped_strings)[1]
        teamBLink = teamBEl[1].find('a').attrs['href']
        teamBScore = teamBEl[0].text
        teamBWin = teamBEl[1].attrs['class'][0].split("-")[1]
        teamB = {'name': teamBName, 'nick': teamBNick, 'link': teamBLink.split("/")[2], 'outcome': teamBWin, 'score': teamBScore}
        # print(raceNum, winningTeam, winningScore, losingTeam, losingScore)
        
        raceList.append({'raceNum': int(raceNum),
                         'round': round,
                         'teamA': teamA,
                         'teamB': teamB,
                         })
    return raceList

In [298]:
def getSailorLinks(reportPage):
    sailorLinks = []
    
    mainTable = reportPage.find('table', class_='results').contents[1]
    allSailors = mainTable.find_all('td', class_='sailor')
    
    for sailor in allSailors:
        sailorLink = 'Unknown'
        if sailor.find('a') != None:
            sailorLink = sailor.find('a').attrs['href'].split("/")[2]
        sailorLinks.append({'name': sailor.text.split(" '")[0].strip(), 'link':sailorLink})
    return sailorLinks

def getTeamReportInfo(reportPage):
    teamInfoList = []
    
    mainTable = reportPage.find('table', class_='results').contents[1]
    teamRows = mainTable.find_all('tr', class_='topborder')
    
    for row in teamRows:
        teamRank = row.contents[1].text
        teamID = row.attrs['class'][2].split("-")[1]
        teamNick = row.find('td', class_='teamname').text
        uniName = row.find('a').text
        teamLink = row.find('a').attrs['href']
        tiebreaker = {'char': row.find('td', class_='tiebreaker').text,'text':row.find('td', class_='tiebreaker').attrs['title']}
        
        teamInfoList.append({'rank': teamRank, 'uniName': uniName, 'teamNick': teamNick, 'teamID': teamID, 'teamLink': teamLink, 'tiebreaker': tiebreaker})
    return teamInfoList

In [299]:
def makeSailorList(sailorData):
    sailorRaceList = []
    
    ports = sailorData.find_all('div', class_='port')
    
    round = ''
    for port in ports:
        round = port.find('h3').text
        tableRows = port.find('table').contents[0].contents
    
        opponentsRow = tableRows[0].find_all('th')
        teamNames = [team.text for team in opponentsRow]
        
        fullTeamNames = [el.find_all('th')[0].text for el in tableRows[1:]]
        teamIDs = [el.attrs['class'][1].split("-")[1] for el in tableRows[1:]]
        
        for tableRow, teamName, fullTeamName, teamID in zip(tableRows[1:], teamNames, fullTeamNames, teamIDs):
            sailorTables = tableRow.find_all("table", class_='tr-boats')
            tempNames = teamNames.copy()
            tempNames.remove(teamName)
            tempFullNames = fullTeamNames.copy()
            tempFullNames.remove(fullTeamName)
            
            tempTeamIDs = teamIDs.copy()
            tempTeamIDs.remove(teamID)
            
            for race, opponent, opponentFull, oppID in zip(sailorTables, tempNames, tempFullNames, tempTeamIDs):
                boats = [list(names.stripped_strings) for names in race.find_all('td')]
                
                for boat in boats:
                    skipperName, skipperYear, crewName, crewYear = ['Unknown'] * 4
                    
                    if len(boat) == 2:
                        skipperName = boat[0].split(" '")[0].strip()
                        skipperYear = boat[0].split(" '")[1].strip()
                        crewName = boat[1].split(" '")[0].strip()
                        crewYear = boat[1].split(" '")[1].strip()
                        
                    sailorRaceList.append({'name': skipperName,
                                           'year': skipperYear,
                                           'pos': 'skipper',
                                           'round': round,
                                           'teamName': teamName, 
                                           'teamID': teamID,
                                           'fullName': fullTeamName, 
                                           'opponent': opponent,
                                           'opponentFull': opponentFull,
                                           'oppID': oppID,
                                           'partner': crewName
                                           })
                    
                    sailorRaceList.append({'name': crewName,
                                           'year': crewYear,
                                           'pos': 'crew',
                                           'round': round,
                                           'teamName': teamName, 
                                           'teamID': teamID,
                                           'fullName': fullTeamName, 
                                           'opponent': opponent,
                                           'opponentFull': opponentFull,
                                           'oppID': oppID,
                                           'partner': skipperName
                                           })
                    
    return sailorRaceList

def getTeamIDs(sailorData):
    data = []
    port = sailorData.find_all('div', class_='port')[0]
    tableRows = port.find('table').contents[0].contents
    opponentsRow = tableRows[0].find_all('th')
    
    for team, el in zip(opponentsRow, tableRows[1:]):   
        teamName = team.text
        uniName = el.find('th').text
        teamID = el.attrs['class'][1].split("-")[1]
        data.append({'teamName': teamName, 'uniName': uniName, 'teamID': teamID})
    
    return data

In [312]:
data = []
totalSailors = {}

for i, regatta in enumerate(list(regattaSoups.keys())):
# regatta = list(regattaSoups.keys())[0]
# regatta = 's24/mcsa-women-team-race'
    print(f"({i + 1}/{len(list(regattas.values()))}) analyzing {regatta}")

    allRaces = regattaSoups[regatta]['allRaces']
    sailors = regattaSoups[regatta]['sailors']
    reportPage = regattaSoups[regatta]['report']
    scoring = regattaSoups[regatta]['scoring']

    if len(allRaces.find_all('table', class_="teamscorelist")) == 0: 
        print(f"no scores entered for {regatta}, skipping")
        # continue

    scoreData = allRaces.find_all('table', class_="teamscorelist")[
        0].contents

    host = allRaces.find("span", itemprop='location').text
    date = allRaces.find("time").attrs['datetime']
    date = date[:10]

    regattaType = allRaces.find("span", itemprop='description').text

    raceRows = [row for table in scoreData for row in table.contents][1:]

    df_races = pd.DataFrame(makeRaceList(raceRows))

    df_sailors = pd.DataFrame(makeSailorList(sailors))
    if len(df_sailors['name'].unique()) < 2:
        print("No rp ented for this regatta", regatta)
        continue

    df_sailorLinks = pd.DataFrame(getSailorLinks(reportPage))

    df_teamReportInfo = pd.DataFrame(getTeamReportInfo(reportPage))
    df_sailorteamInfo = pd.DataFrame(getTeamIDs(sailors))

    # merge dfs 
    df_sailors = pd.merge(df_sailors, df_sailorLinks, how='left', on='name')
    # df_totalSailors = pd.concat([df_totalSailors, df_sailors])

    for _, race_result in df_races.iterrows():
        raceNum = race_result['raceNum']
        round = race_result['round']
        
        teamA = race_result['teamA']
        teamAID = df_teamReportInfo.loc[df_teamReportInfo['uniName'] == teamA['name'], 'teamID'].iat[0]
        teamAName = df_sailorteamInfo.loc[df_sailorteamInfo['teamID'] == teamAID, 'teamName'].iat[0]
        
        teamB = race_result['teamB']
        teamBID = df_teamReportInfo.loc[df_teamReportInfo['uniName'] == teamB['name'], 'teamID'].iat[0]
        teamBName = df_sailorteamInfo.loc[df_sailorteamInfo['teamID'] == teamBID, 'teamName'].iat[0]

        teamASailors = df_sailors.loc[(df_sailors['round'] == round) & (df_sailors['teamID'] == teamAID) & (df_sailors['oppID'] == teamBID)]
        teamABoats = []
        if len(teamASailors) > 0:
            for i in range(0,6,2):
                skipperName = teamASailors.iloc[i]['name']
                skipperYear = teamASailors.iloc[i]['year']
                skipperLink = teamASailors.iloc[i]['link']
                skipperKey = skipperLink if skipperLink != 'Unknown' else skipperName + "-" + teamAName
                
                totalSailors[skipperKey] = {'name': skipperName, 'year': skipperYear, 'link': skipperLink, 'key': skipperKey}
                
                crewName = teamASailors.iloc[i + 1]['name']
                crewYear = teamASailors.iloc[i + 1]['year']
                crewLink = teamASailors.iloc[i + 1]['link']
                crewKey = crewLink if crewLink != 'Unknown' else crewName + "-" + teamAName
                
                totalSailors[crewKey] = {'name': crewName, 'year': crewYear, 'link': crewLink, 'key': crewKey}
                
                teamABoats.append({'skipperName': skipperName, 'skipperLink': skipperLink,'skipperKey': skipperKey, 
                                'crewName': crewName, 'crewLink': crewLink, 'crewKey': crewKey})
        
        teamBSailors = df_sailors.loc[(df_sailors['round'] == round) & (df_sailors['teamID'] == teamBID) & (df_sailors['oppID'] == teamAID)]
        teamBBoats = []

        if len(teamBSailors) > 0:
            for i in range(0,6,2):
                skipperName = teamBSailors.iloc[i]['name']
                skipperYear = teamBSailors.iloc[i]['year']
                skipperLink = teamBSailors.iloc[i]['link']
                skipperKey = skipperLink if skipperLink != 'Unknown' else skipperName + "-" + teamBName
                
                totalSailors[skipperKey] = {'name': skipperName, 'year': skipperYear, 'link': skipperLink, 'key': skipperKey}
                
                crewName = teamBSailors.iloc[i + 1]['name']
                crewYear = teamBSailors.iloc[i + 1]['year']
                crewLink = teamBSailors.iloc[i + 1]['link']
                crewKey = crewLink if crewLink != 'Unknown' else crewName + "-" + teamBName
                
                totalSailors[crewKey] = {'name': crewName, 'year': crewYear, 'link': crewLink, 'key': crewKey}
                
                teamBBoats.append({'skipperName': skipperName, 'skipperLink': skipperLink,'skipperKey': skipperKey, 
                                'crewName': crewName, 'crewLink': crewLink, 'crewKey': crewKey})
        
        
        data.append({'raceID': f"{regatta}/{raceNum}",
                    'raceNum': raceNum, 'round': round,
                    'teamAName': teamAName,
                    'teamAUni': teamA['name'],
                    'teamANick': teamA['nick'],
                    'teamALink': teamA['link'],
                    'teamAID': teamAID,
                    'teamABoats': teamABoats,
                    'teamAScore': teamA['score'],
                    'teamAOutcome': teamA['outcome'],
                    'teamBName': teamBName,
                    'teamBUni': teamB['name'],
                    'teamBNick': teamB['nick'],
                    'teamBLink': teamB['link'],
                    'teamBID': teamBID,
                    'teamBBoats': teamBBoats,
                    'teamBScore': teamB['score'],
                    'teamBOutcome': teamB['outcome'],
                    })
        

df_final = pd.DataFrame(data)

(1/73) analyzing s24/icsa-open-team-racing-national
(2/73) analyzing s24/icsa-women-team-race-national
(3/73) analyzing s24/b10-team-race
(4/73) analyzing s24/mcsa-women-team-race
(5/73) analyzing s24/saisa-spring-women-team-race-championships
(6/73) analyzing s24/2024-legler
(7/73) analyzing s24/carter-ford-open-pccsc-team-race
(8/73) analyzing s24/maisa-women-team-race-championships
(9/73) analyzing s24/thompson-camel-site
(10/73) analyzing s24/thompson-cga
(11/73) analyzing s24/mcsa-open-team-race-qualifier
(12/73) analyzing s24/saisa-spring-team-race-championships
(13/73) analyzing s24/prosser
(14/73) analyzing s24/neisa-open-team-race-fowle
(15/73) analyzing s24/pccsc-women-team-race
(16/73) analyzing s24/seisa-women-team-race
(17/73) analyzing s24/seisa-open-team-racing
(18/73) analyzing s24/yale-women-team-race
(19/73) analyzing s24/mystic-lake-team-race
(20/73) analyzing s24/alymers-team-race-invite
(21/73) analyzing s24/bowdoin-women-team-race
(22/73) analyzing s24/41st-silver

In [313]:
df_totalSailors = pd.DataFrame.from_dict(totalSailors,orient='index')
df_totalSailors = df_totalSailors.reset_index(drop=True)
df_totalSailors.to_json("trSailorInfo.json", index=False)
df_totalSailors

,name,year,link,key
0,Jack Egan,25,jack-egan,jack-egan
1,Elle Sykes,27,elle-sykes,elle-sykes
2,Morgan Pinckney,27,morgan-pinckney,morgan-pinckney
3,Jessi Avila-Shah,25,jessi-avila-shah,jessi-avila-shah
4,Teddy Nicolosi,24,teddy-nicolosi,teddy-nicolosi
...,...,...,...,...
1093,Jack Blackman,24,jack-blackman,jack-blackman
1094,Genaro Flores - Machorro,25,genaro-flores-machorro,genaro-flores-machorro
1095,Maggie McEachen,27,maggie-mceachen,maggie-mceachen
1096,Aki Schuessler-Langeheine,24,aki-schuessler-langeheine,aki-schuessler-langeheine


Brainstorming on datastructure: 
```
# By sailor
#columns = ['raceNum', 'round', 'name','link', 'pos', 'partner', 'teamName', 'fullTeamName', 'teamNick','teamLink', 'teamScore', 'win?', 'opponentTeamName','opponentFullName', 'opponentNick','opponentLink', 'opponentScore']

# By race
# potentially better because we need it by race for the elo calc anyways, and it will always be displayed by race in the regatta page. 
# downside is that it makes it harder to display on an individual sailor page, but the elo calc can handle that
# columns = ['raceNum', 'round',
#            'winTeamName', 'winTeamUni', 'winTeamNick', 'winTeamLink', 'winTeamID', 'winTeamBoatA', 'winTeamBoatB', 'winTeamBoatC','winTeamScore'
#            'loseTeamName', 'loseTeamUni', 'loseTeamNick', 'loseTeamLink', 'loseTeamID', 'loseTeamBoatA', 'loseTeamBoatB', 'loseTeamBoatC','loseTeamScore']
# boat = ['skipperName', 'skipperLink', 'crewName', 'crewLink']```

In [286]:
df_final.to_json("s24teamraces.json", index=False)